In [ ]:
from pyspark.sql import *;
from pyspark.sql.functions import *;
from pyspark.sql.window import *;
import pandas as pd;

In [ ]:
spark = SparkSession.builder.appName("Q2").getOrCreate()

In [ ]:
df = spark.read.csv("Sales Data.csv",header=True,inferSchema=True)
df.printSchema()
df.show()

root
 |-- _c0: integer (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Sales: double (nullable = true)
 |-- City: string (nullable = true)
 |-- Hour: integer (nullable = true)

+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|_c0|Order ID|             Product|Quantity Ordered|Price Each|      Order Date|    Purchase Address|Month| Sales|          City|Hour|
+---+--------+--------------------+----------------+----------+----------------+--------------------+-----+------+--------------+----+
|  0|  295665|  Macbook Pro Laptop|               1|    1700.0|30-12-2019 00:01|136 Church St, Ne...|   12|1700.0| New York City|   0|
|  1|  29

In [ ]:
df.select([count(when(isnan(c)|isnull(c),c)).alias(c) for c in df.columns]).show()

+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|_c0|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|Month|Sales|City|Hour|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|  0|       0|      0|               0|         0|         0|               0|    0|    3|   0|   0|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+



In [ ]:
numerical_cols = ["Sales"]
for c in numerical_cols:
  mean_value = df.select(mean(c)).first()[0]
  df = df.fillna({c:mean_value})

df.select([count(when(isnan(c)|isnull(c),c)).alias(c) for c in df.columns]).show()

+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|_c0|Order ID|Product|Quantity Ordered|Price Each|Order Date|Purchase Address|Month|Sales|City|Hour|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+
|  0|       0|      0|               0|         0|         0|               0|    0|    0|   0|   0|
+---+--------+-------+----------------+----------+----------+----------------+-----+-----+----+----+



In [ ]:
df = df.filter(col("Order ID").isNotNull() & col("Product").isNotNull())

In [ ]:
df = df.dropDuplicates()

In [ ]:
df.printSchema()

root
 |-- _c0: integer (nullable = true)
 |-- Order ID: integer (nullable = true)
 |-- Product: string (nullable = true)
 |-- Quantity Ordered: integer (nullable = true)
 |-- Price Each: double (nullable = true)
 |-- Order Date: string (nullable = true)
 |-- Purchase Address: string (nullable = true)
 |-- Month: integer (nullable = true)
 |-- Sales: double (nullable = false)
 |-- City: string (nullable = true)
 |-- Hour: integer (nullable = true)



In [ ]:
df = df.filter((col("Sales")>=0) & (col("Price Each")>=0) & (col("Quantity Ordered")>=0))

In [ ]:
product_sales = df.groupBy("Product").agg(round(sum("Sales"),2).alias("Product Sales"))
product_sales.show()

+--------------------+-------------+
|             Product|Product Sales|
+--------------------+-------------+
|    Wired Headphones|    246651.93|
|  Macbook Pro Laptop|    8037600.0|
|Apple Airpods Hea...|    2349150.0|
|              iPhone|    4794300.0|
|Lightning Chargin...|    347094.15|
|Bose SoundSport H...|   1345565.43|
|USB-C Charging Cable|    286674.79|
|AAA Batteries (4-...|     92740.83|
|        20in Monitor|    454148.71|
|    27in FHD Monitor|    1132424.5|
|     Vareebadd Phone|     827200.0|
|34in Ultrawide Mo...|   2355558.01|
|            LG Dryer|     387600.0|
|AA Batteries (4-p...|    106300.05|
|        Google Phone|    3319200.0|
|       Flatscreen TV|    1445700.0|
|  LG Washing Machine|     399600.0|
|27in 4K Gaming Mo...|   2435097.56|
|     ThinkPad Laptop|    4129958.7|
+--------------------+-------------+

